In [1]:
import re
import numpy as np
import pandas as pd
import glob

In [58]:
path = "/Volumes/SSD/00_Data_Processing/Pre_processed/02_Individuals_Colliders/" 

  
# csv files in the path
files = glob.glob(path + "/*.csv")
  
# defining an empty list to store 
# content
data_frame = pd.DataFrame()
content = []
  
# checking all the csv files in the 
# specified path
for filename in files:
    
    # reading content of csv file
    # content.append(filename)
    df = pd.read_csv(filename)
    content.append(df)
    print(filename)
  
# converting content to data frame
data_frame = pd.concat(content)


/Volumes/SSD/00_Data_Processing/Pre_processed/02_Individuals_Colliders/8469.csv
/Volumes/SSD/00_Data_Processing/Pre_processed/02_Individuals_Colliders/8673.csv
/Volumes/SSD/00_Data_Processing/Pre_processed/02_Individuals_Colliders/8695.csv
/Volumes/SSD/00_Data_Processing/Pre_processed/02_Individuals_Colliders/9472.csv
/Volumes/SSD/00_Data_Processing/Pre_processed/02_Individuals_Colliders/9502.csv
/Volumes/SSD/00_Data_Processing/Pre_processed/02_Individuals_Colliders/9601.csv


In [59]:
data_frame.columns

Index(['Unnamed: 0', 'index', 'SubjectID', 'Session', 'SessionSubsection',
       'timeStampDataPointStart', 'timeStampDataPointEnd',
       'timeStampGetVerboseData', 'combinedGazeValidityBitmask',
       'rayCastHitsCombinedEyes', 'eyePositionCombinedWorld.x',
       'eyePositionCombinedWorld.y', 'eyePositionCombinedWorld.z',
       'eyeDirectionCombinedWorld.y', 'eyeDirectionCombinedWorld.z',
       'eyeDirectionCombinedLocal.x', 'eyeDirectionCombinedLocal.y',
       'eyeDirectionCombinedLocal.z', 'hmdPosition.x', 'hmdPosition.y',
       'hmdPosition.z', 'hmdDirectionForward.x', 'hmdDirectionForward.y',
       'hmdDirectionForward.z', 'hmdRotation.x', 'hmdRotation.y',
       'hmdRotation.z', 'hmdDirectionUp.x', 'hmdDirectionUp.y',
       'hmdDirectionUp.z', 'playerBodyPosition.x', 'playerBodyPosition.y',
       'playerBodyPosition.z', 'bodyTrackerPosition.x',
       'bodyTrackerPosition.y', 'bodyTrackerPosition.z',
       'bodyTrackerRotation.x', 'bodyTrackerRotation.y',
       'bod

In [60]:
data_frame.drop(['Unnamed: 0', 'rayCastHitsCombinedEyes', 'timeStampGetVerboseData', 'hitObjectColliderBoundsCenter'],axis=1, inplace=True)

In [61]:
#Since we have two colliders hits per frame, we calculate the distance between each hit and the participant
data_frame['Eucledian_distance'] = np.linalg.norm(data_frame.loc[:, ["hitPointOnObject_x","hitPointOnObject_y","hitPointOnObject_z"]].values - data_frame.loc[:, ["playerBodyPosition.x","playerBodyPosition.y","playerBodyPosition.z"]], axis=1)

In [62]:
# Since the collider names are too detailed, here we create a dictionary with patterns to classify them into our categories of interest
# INCLUDE CHURCH
patterns = {'\d{2}_Sa':'Passive_Agent', '\d{2}_Cma':'Active_Agent', 'Building_\d+': 'Building'}
patterns.update(dict.fromkeys(['Castle-TaskBuilding_56','HighSilo-TaskBuilding_49', 'Windmill-TaskBuilding_10_1', 'Church_TaskBuilding_16'], 'Global_Landmark'))
patterns.update(dict.fromkeys(['TaskBuilding_2','TaskBuilding_3', 'TaskBuilding_5', 'TaskBuilding_8', 'TaskBuilding_9', 'TaskBuilding_11', 'TaskBuilding_13', 'TaskBuilding_14', 'TaskBuilding_20', 'TaskBuilding_21', 'TaskBuilding_23','TaskBuilding_27', 'TaskBuilding_29', 'TaskBuilding_32', 'TaskBuilding_34',  'TaskBuilding_38', 'TaskBuilding_41', 'TaskBuilding_42', 'TaskBuilding_44', 'TaskBuilding_45', 'TaskBuilding_47', 'TaskBuilding_50', 'TaskBuilding_51', 'TaskBuilding_52', 'BasketballCourt_58', 'Construction_57', 'Graffity_02', 'Graffity_03', 'Graffity_05', 'Graffity_08', 'Graffity_09', 'Graffity_11', 'Graffity_13', 'Graffity_14', 'Graffity_20', 'Graffity_21', 'Graffity_23', 'Graffity_27', 'Graffity_29', 'Graffity_32', 'Graffity_34', 'Graffity_38', 'Graffity_41', 'Graffity_42', 'Graffity_44', 'Graffity_45', 'Graffity_47',  'Graffity_50', 'Graffity_51', 'Graffity_52'], 'TaskBuilding_Public'))
patterns.update(dict.fromkeys(['TaskBuilding_1','TaskBuilding_4', 'TaskBuilding_6', 'TaskBuilding_7', 'TaskBuilding_12', 'TaskBuilding_15', 'TaskBuilding_17', 'TaskBuilding_18', 'TaskBuilding_19', 'TaskBuilding_22', 'TaskBuilding_24','TaskBuilding_25', 'TaskBuilding_26', 'TaskBuilding_28', 'TaskBuilding_30',  'TaskBuilding_31', 'TaskBuilding_33', 'TaskBuilding_35', 'TaskBuilding_36', 'TaskBuilding_37', 'TaskBuilding_39', 'TaskBuilding_40', 'TaskBuilding_43', 'TaskBuilding_48', 'TaskBuilding_54','TaskBuilding_55','Graffity_1','Graffity_4', 'Graffity_6', 'Graffity_7', 'Graffity_12', 'Graffity_15', 'Graffity_17', 'Graffity_18', 'Graffity_19', 'Graffity_22', 'Graffity_24','Graffity_25', 'Graffity_26', 'Graffity_28', 'Graffity_30',  'Graffity_31', 'Graffity_33', 'Graffity_35', 'Graffity_36', 'Graffity_37', 'Graffity_39', 'Graffity_40', 'Graffity_43', 'Graffity_48', 'Graffity_54', 'Graffity_55' ], 'TaskBuilding_Residential'))
default_val = 'Background'

In [63]:
# Here we look for the patterns contained in the dictionary and create the more general/informative variable Collider_Categorical
data_frame['Collider_Categorical'] = data_frame['hitObjectColliderName'].apply(lambda x: next((val for key, val in patterns.items() if re.match(key, x)), default_val))

In [64]:
# Since the double hits where unfolded in a long format in case of more than one hit per frame the second hit would be the row directly underneath.
# Therefore, if we want to know which hit was closest to the participant per frame we can compare each row with the previous one and see which one has the smallest Euclidean distance.
data_frame['Previous_Euclidean_value'] = data_frame['Eucledian_distance'].shift(1)
#Here we declare the conditions to choose between collider hits:
data_frame['Collider_stays'] = (data_frame["ordinalOfHit"] == 2) & (data_frame['Collider_Categorical'] != 'Background') & (data_frame['Eucledian_distance'] <  data_frame['Previous_Euclidean_value']) 

In [65]:
original_shape = data_frame.shape

In [66]:
data_frame.reset_index(inplace=True)
#Drop all the second hit colliders that do not comply with the criteria
indexCollider = data_frame[(data_frame["ordinalOfHit"] == 2) & (data_frame['Collider_stays'] == False)].index

In [67]:
depleted_data = data_frame.drop(index=indexCollider)

In [68]:
depleted_data.reset_index(inplace=True, drop=True)
#Take the index of all second colliders that will stay
indexColliderStays = depleted_data[depleted_data['Collider_stays'] == True].index
#We subtract one from that list of indexes because now is the first collider that has to go (so row directly on top)
indexColliderDelete = indexColliderStays - 1
depleted_data_1 = depleted_data.drop(indexColliderDelete)

In [ ]:
#Create variable that differenciates body from face hits on agents
depleted_data_1["Face_Hits"] = "Not_Agent"
mask_Face = ((depleted_data_1["Collider_Categorical"].str.contains(pat="_Agent")) & (depleted_data_1["hitColliderType"] == "UnityEngine.SphereCollider"))
mask_Body = ((depleted_data_1["Collider_Categorical"].str.contains(pat="_Agent")) & (depleted_data_1["hitColliderType"] == "UnityEngine.BoxCollider"))
depleted_data_1.loc[mask_Face, 'Face_Hits'] = "Face"
depleted_data_1.loc[mask_Body, 'Face_Hits'] = "Body"
depleted_data_1["Face_Hits"].value_counts()

In [69]:
depleted_data_1.to_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/clean_seven_participants_3.csv")

In [ ]:
Make_Error

## Concat the clean data sets

In [2]:
clean_data = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/clean_seven_participants.csv")
clean_data_1 = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/clean_seven_participants_1.csv")
clean_data_2 = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/clean_seven_participants_2.csv")
clean_data_3 = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/clean_seven_participants_3.csv")

In [3]:
dataframes = [clean_data, clean_data_1,  clean_data_2, clean_data_3]
for df in dataframes:
    df.drop(["Unnamed: 0", "level_0", 'Previous_Euclidean_value'], axis=1, inplace=True)

In [4]:
result = pd.concat(dataframes)

In [5]:
result.head()

,index,SubjectID,Session,SessionSubsection,timeStampDataPointStart,timeStampDataPointEnd,combinedGazeValidityBitmask,eyePositionCombinedWorld.x,eyePositionCombinedWorld.y,eyePositionCombinedWorld.z,...,bodyTrackerRotation.z,hitColliderType,hitObjectColliderName,ordinalOfHit,hitPointOnObject_x,hitPointOnObject_y,hitPointOnObject_z,Eucledian_distance,Collider_Categorical,Collider_stays
0,121419,479,1,1,1.652967e+09,1.652967e+09,3,-59.122986,2.413162,34.732815,...,55.068886,UnityEngine.MeshCollider,TaskBuilding_35,1.0,-63.126041,2.384691,44.754692,10.849845,TaskBuilding_Public,False
1,121421,479,1,1,1.652967e+09,1.652967e+09,3,-59.122986,2.413162,34.732815,...,55.068886,UnityEngine.MeshCollider,TaskBuilding_35,1.0,-63.126041,2.384691,44.754692,10.849845,TaskBuilding_Public,False
2,121423,479,1,1,1.652967e+09,1.652967e+09,3,-59.119831,2.413259,34.732769,...,55.068878,UnityEngine.MeshCollider,TaskBuilding_35,1.0,-63.102364,2.409164,44.764919,10.853945,TaskBuilding_Public,False
3,121425,479,1,1,1.652967e+09,1.652967e+09,3,-59.135059,2.412430,34.733551,...,55.070137,UnityEngine.MeshCollider,TaskBuilding_35,1.0,-63.168682,2.391426,44.736275,10.849811,TaskBuilding_Public,False
4,121427,479,1,1,1.652967e+09,1.652967e+09,3,-59.135059,2.412407,34.733528,...,55.071308,UnityEngine.MeshCollider,TaskBuilding_35,1.0,-63.168106,2.389861,44.736523,10.849602,TaskBuilding_Public,False


# Separate face and body hits

In [6]:
result["Face_Hits"] = "Not_Agent"
#Our face collider can be identified because they have a collider type Sphere while the body was captured by a boxcollider
mask_Face = ((result["Collider_Categorical"].str.contains(pat ="_Agent")) & (result["hitColliderType"] == "UnityEngine.SphereCollider"))
mask_Body = ((result["Collider_Categorical"].str.contains(pat ="_Agent")) & (result["hitColliderType"] == "UnityEngine.BoxCollider"))
result.loc[mask_Face, 'Face_Hits'] = "Face"
result.loc[mask_Body, 'Face_Hits'] = "Body"

In [7]:
result["Face_Hits"].value_counts()

Not_Agent    25641663
Body           570296
Face           140425
Name: Face_Hits, dtype: int64

## Create continuous time

In [8]:
# Group by Subject and Session because the continuous time only makes sence on a session scale
all_participants_sessions = pd.DataFrame(result.groupby(["SubjectID", "Session"])["hitColliderType"].count())
all_participants_sessions.reset_index(inplace=True)
# Create tuples to later subset data to subject-session scale. We create this because subjects can have a different number of sessions, in this way we only subset for the existing pairs S-S.
L_all_participants_sessions = list(zip(all_participants_sessions.SubjectID, all_participants_sessions.Session))

In [9]:
len(L_all_participants_sessions)

127

In [ ]:
temporals = []
for tuple in L_all_participants_sessions:
    subject, session = tuple
    temporal = result[(result['SubjectID'] == subject) & (result['Session'] == session)]
    temporal_c = temporal.copy(deep=True)
    temporal_c["Continuous_Time"] = np.round((temporal_c.iloc[:, 5] - temporal_c.iloc[0,5])/60, 3)
    temporals.append(temporal_c)
    print(tuple)

In [11]:
data_frames_list = pd.concat(temporals, axis=0)

In [12]:
data_frames_list.tail()

,index,SubjectID,Session,SessionSubsection,timeStampDataPointStart,timeStampDataPointEnd,combinedGazeValidityBitmask,eyePositionCombinedWorld.x,eyePositionCombinedWorld.y,eyePositionCombinedWorld.z,...,hitObjectColliderName,ordinalOfHit,hitPointOnObject_x,hitPointOnObject_y,hitPointOnObject_z,Eucledian_distance,Collider_Categorical,Collider_stays,Face_Hits,Continuous_Time
6001272,1897591,9601,5,3,1.652798e+09,1.652798e+09,3,-110.984283,4.037135,103.240768,...,Building_210,1.0,-295.833679,4.772648,-74.126945,256.299333,Building,False,Not_Agent,30.912
6001273,1897593,9601,5,3,1.652798e+09,1.652798e+09,3,-111.037956,4.038515,103.227974,...,Building_210,1.0,-295.258362,5.042946,-74.824463,256.321150,Building,False,Not_Agent,30.912
6001274,1897595,9601,5,3,1.652798e+09,1.652798e+09,3,-111.037956,4.038515,103.227974,...,Building_210,1.0,-295.258362,5.042946,-74.824463,256.321150,Building,False,Not_Agent,30.912
6001275,1897597,9601,5,3,1.652798e+09,1.652798e+09,3,-111.081200,4.038269,103.201927,...,Building_213,1.0,-276.617615,5.832996,-48.707134,224.754887,Building,False,Not_Agent,30.912
6001276,1897599,9601,5,3,1.652798e+09,1.652798e+09,3,-111.081200,4.038269,103.201927,...,Building_213,1.0,-276.617615,5.832996,-48.707134,224.754887,Building,False,Not_Agent,30.912


In [13]:
data_frames_list.to_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/Data_Complete.csv")

# If we want to keep only the perfect cases run this 

In [ ]:
clean_data = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/clean_seven_participants.csv")
The_perfect_Set = clean_data[(clean_data['SubjectID'] == 1754) | (clean_data['SubjectID'] == 2258) | (clean_data['SubjectID'] == 2693)]

In [ ]:
clean_data_1 = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/clean_seven_participants_1.csv")
The_perfect_Set_1 = clean_data_1[(depleted_data['SubjectID'] == 4176) | (clean_data_1['SubjectID'] == 4796) | (clean_data_1['SubjectID'] == 4917) | (clean_data_1['SubjectID'] == 5238)]

In [ ]:
clean_data_2 = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/clean_seven_participants_2.csv")
The_perfect_Set_2 = clean_data_2[(clean_data_1['SubjectID'] == 6642) | (clean_data_2['SubjectID'] == 7412) | (clean_data_2['SubjectID'] == 7842)| (clean_data_2['SubjectID'] == 8007)]

In [ ]:
clean_data_3 = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/clean_seven_participants_3.csv")
The_perfect_Set_3 = clean_data_3[(clean_data_3['SubjectID'] == 8469) | (clean_data_3['SubjectID'] == 8673) | (clean_data_3['SubjectID'] == 9472)| (clean_data_3['SubjectID'] == 9601)]